# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
raw_df = pd.read_csv('../output_data/WeatherPy_output/raw_weather.csv')
print(len(raw_df))
raw_df.head()

559


,City Name,City Latitude,City Longitude,Max Tempurature,Humidity,Cloud coverage,Wind speed,City Country,City datetime
0,Avarua,-21.2078,-159.7750,78.800,83.0,27.0,6.20,CK,1.609868e+09
1,Ancud,-41.8697,-73.8203,68.000,52.0,0.0,4.10,CL,1.609868e+09
3,Montepuez,-13.1256,38.9997,70.574,94.0,100.0,0.62,MZ,1.609868e+09
5,Port Moresby,-9.4431,147.1797,77.198,88.0,90.0,2.08,PG,1.609868e+09
6,Cape Town,-33.9258,18.4232,73.004,64.0,20.0,9.80,ZA,1.609868e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
import gmaps
gmaps.configure(api_key=g_key)

In [5]:
coords = raw_df[['City Latitude', 'City Longitude']].fillna(0)
humidity = raw_df['Humidity']

fig = gmaps.figure(map_type='HYBRID')

heatmap_layer = gmaps.heatmap_layer(coords, weights=humidity, dissipating=False, max_intensity=10, point_radius=1)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_temp_mask = (raw_df['Max Tempurature'] < 80) & (raw_df['Max Tempurature'] > 70)
ideal_wind_mask = raw_df['Wind speed'] < 10
ideal_cloud_mask = raw_df['Cloud coverage'] == 0 

ideal_spots_df = raw_df[ideal_temp_mask & ideal_wind_mask & ideal_cloud_mask]
ideal_spots_df.index = ideal_spots_df['City Name']
ideal_spots_df = ideal_spots_df.rename(columns={'City Name': 'City'})
ideal_spots_df.head()

,City,City Latitude,City Longitude,Max Tempurature,Humidity,Cloud coverage,Wind speed,City Country,City datetime
City Name,,,,,,,,,
Saint-Pierre,Saint-Pierre,-21.3393,55.4781,78.80,61.0,0.0,3.10,RE,1.609868e+09
Saint-Philippe,Saint-Philippe,-21.3585,55.7679,78.80,61.0,0.0,3.10,RE,1.609868e+09
Bandarbeyla,Bandarbeyla,9.4942,50.8122,74.12,65.0,0.0,9.51,SO,1.609868e+09
Coyhaique,Coyhaique,-45.5752,-72.0662,75.20,29.0,0.0,4.10,CL,1.609869e+09
Raja,Raja,8.4596,25.6780,79.16,22.0,0.0,2.26,SS,1.609869e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
ideal_coords = ideal_spots_df[['City Latitude','City Longitude']]
city_names = ideal_spots_df['City']
string_coords = []
responses = []

for vacation_spot in range(len(ideal_coords)):
    str_coord = f'{str(ideal_coords.iloc[vacation_spot, 0])}, {str(ideal_coords.iloc[vacation_spot,1])}'
    string_coords.append(str_coord)

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target_search = "Hotel"
target_radius = 5000
target_type = "hotel"
        
for index, vacation_spot in enumerate(string_coords): 
    print(f'Searching for hotels within {target_radius}m of {city_names[index]} with Google Places API')
    
    params = {
        "location": vacation_spot,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    response = requests.get(base_url, params=params)
    responses.append(response)

Searching for hotels within 5000m of Saint-Pierre with Google Places API
Searching for hotels within 5000m of Saint-Philippe with Google Places API
Searching for hotels within 5000m of Bandarbeyla with Google Places API
Searching for hotels within 5000m of Coyhaique with Google Places API
Searching for hotels within 5000m of Raja with Google Places API
Searching for hotels within 5000m of Salalah with Google Places API
Searching for hotels within 5000m of Saint-Joseph with Google Places API
Searching for hotels within 5000m of Saint-Leu with Google Places API
Searching for hotels within 5000m of Manthani with Google Places API
Searching for hotels within 5000m of Port Blair with Google Places API
Searching for hotels within 5000m of Birao with Google Places API
Searching for hotels within 5000m of Bilma with Google Places API
Searching for hotels within 5000m of Cockburn Town with Google Places API
Searching for hotels within 5000m of Eilat with Google Places API
Searching for hotels w

In [8]:
clean_responses = [resp for resp in responses if resp.json()['status'] != 'ZERO_RESULTS']

In [14]:
from citipy import citipy

# Getting coords of the FIRST HOTEL OF EACH SEARCH WITHIN 5000m OF CITY 
hotel_info = {'Hotel Name': [],
              'Lat': [],
              'Lng': [],
              'City': []}
    
for result_list in clean_responses:
    hotels = result_list.json()
    first_hotel = hotels['results'][0]
    
    hotel_info['Hotel Name'].append(first_hotel['name'])
    hotel_info['Lat'].append(first_hotel['geometry']['location']['lat'])
    hotel_info['Lng'].append(first_hotel['geometry']['location']['lng'])

# Getting cities of hotels (hotel may not be within the city of search)
for coord_pair in zip(hotel_info['Lat'], hotel_info['Lng']):
    city = citipy.nearest_city(coord_pair[0], coord_pair[1]).city_name
    hotel_info['City'].append(city.title())

In [15]:
# Make hotel_df w/ hotel_name, lat, lng, country, city_name cols
countries = ideal_spots_df[['City','City Country']]

hotel_df = pd.DataFrame(hotel_info)
hotel_df = pd.merge(hotel_df, countries, on='City',how='inner')
hotel_df = hotel_df.rename(columns={'City Country': 'Country'})

try:
    hotel_df = hotel_df.sample(10)
except:
    hotel_df = hotel_df

hotel_df

,Hotel Name,Lat,Lng,City,Country
0,"Villa Delisle, Hotel & Spa",-21.343490,55.473758,Saint-Pierre,RE
1,Pinpin d'Amour,-21.359632,55.720716,Saint-Philippe,RE
2,JABIR HOTEL,9.497514,50.810636,Bandarbeyla,SO
3,InterCityHotel Salalah,17.020416,54.093792,Salalah,OM
4,Village Sauvage,-21.379092,55.659249,Saint-Joseph,RE
5,Iloha Seaview Hotel,-21.148180,55.277680,Saint-Leu,RE
6,Welcomhotel Bay Island Port Blair,11.676177,92.740200,Port Blair,IN
7,Osprey Beach Hotel,21.459754,-71.147445,Cockburn Town,TC
8,Abaco Beach Resort,26.546252,-77.046387,Marsh Harbour,BS


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map

hotel_names = hotel_df['Hotel Name'].values.tolist()

marker_layer = gmaps.marker_layer(locations, hotel_names, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

I thought it was interesting that 4 hotels were 'reccomended' in such close proximity.